## This notebook demonstrates the 8 steps to create and export an invasive tree species image
> #### Input: Reference points (GEE FeatureCollection) containing the classes of interest as a property.
> #### Arguments: A start and end data, cloud threshold, area of interest, image composite interval
> #### Output: Invasive Tree species map

**Step 1**: Load modules  
**Step 2**: Prepare time series composite and additional covariates  
**Step 3**: Prepare dataset for model training and evaluation  
**Step 4**: Export data (optional)  
**Step 5**: Fit model  
**Step 6**: Model evaluation  
**Step 7**: Visualize classified image  
**Step 8**: Export classified image  

## Step 1: Load modules

In [1]:
%load_ext watermark

In [13]:
import ee
# ee.Authenticate()
ee.Initialize()
import geemap

# Add module to environment varibles
import sys
MODULE_FULL_PATH = r'C:\Users\coach\myfiles\postdoc\code\Invasive_Species_Mapping\code'
sys.path.insert(1, MODULE_FULL_PATH)

# Load python modules with preprocessing functions. 
from timeSeriesFunctions import prepareTS
from covariateFunctions import prepareCovariates
from trainDataFunctions import prepareTrainingData
from modelFitFunctions import prepareModel
from geeml.utils import eeprint

In [3]:
%watermark -v -m --iversions

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.5.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

geemap: 0.19.6
ee    : 0.1.327
sys   : 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]



In [4]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Step 2: Prepare time series composite

### Prepare Sentinel-2 Data

In [7]:
# start and end dates for time series
Date_Start = ee.Date('2017-01-01');
Date_End = ee.Date('2017-12-31');

# how many days to summarise in each image e.g 30 days = 12 images per year
day_int = 30 #step size

# cloud probabillity threshold
CLOUD_THRESH=30

# which bands to keep
BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6','B7', 'B8', 'B8A', 'B9', 'B11','B12', 'ndvi','ndwi','ndre','nbr','evi']

aoi = ee.Geometry.Point([25.181, -28.490])
# define your study area
study_area1 = ee.Geometry.Polygon([[[30.555572509765607, -24.407014032416797],
          [31.065750122070295, -24.40888986482876],
          [31.065750122070295, -24.40888986482876],
          [31.049957275390607, -25.121539326337704],
          [30.560379028320295, -25.133972597740186],
          [30.555572509765607, -24.407014032416797]]], None, False)

# flatten imagecollection to image - each image becomes a band
s2TS = prepareTS(Date_Start,Date_End,day_int,study_area1,CLOUD_THRESH,BANDS)\
.timeSeries(satellite = 'S2', s2_level = 1)

# Add additional terrain related covariates
X = prepareCovariates(proj = 'EPSG:32726', covariates = s2TS, nAngles = 10).addCovariates(rotatedCoords = True, topoBands =True)
X.bandNames()

In [8]:
Map.addLayer(study_area1)
Map.centerObject(study_area1, 8)

## Step 3: Prepare dataset for model training and evaluation

### Add additional covariates and extract covariates to points.

#### Load training data

In [9]:
# Import reference points
points = ee.FeatureCollection("users/moilwekk/merged_classes")

# name of the label property
label = 'code'

# number of folds for kfold
FOLDS = 10       

# Extract covariate image values to points
pTrainData = prepareTrainingData(covariates = X, points = points, targetProperty = label, nFolds = FOLDS, proj =  'EPSG:32726')
train = pTrainData.covariatesToPoints()
train.limit(2)

## Step 4: Export data (Optional)

### Export training data and image to run inference on

In [13]:
# export training data and image to classify
# Define the export parameters
export_params1 = {
    'collection': train,
    'description': 'export_trainData',
    'assetId': 'users/geethensingh/trainData'
}

export_params2 = {
    'image': X,
    'description': 'export_inferenceImage',
    'assetId': 'users/geethensingh/inferenceImage',
    'scale': 10,
    'region': study_area1
}

# Export the data to the Earth Engine asset
ee.batch.Export.table.toAsset(**export_params1).start()
ee.batch.Export.image.toAsset(**export_params2).start()

## Step 5: Fit model

### Fit a random forest model using Kfold cross validation. Each fold is one spatial cluster (clustering on x and y coordinates)

In [12]:
# load DATA (if exported earlier)
# train = ee.FeatureCollection('users/geethensingh/trainData')
# X = ee.Image('users/geethensingh/inferenceImage')

bandnames = X.bandNames()
# number of classes
NCLASS = train.aggregate_histogram(label).size()

In [17]:
# fit k-fold classifiers
# apply function and train kfold classifieres
result = prepareModel(dataset = train, responseCol = label, inferenceImage = X, bandNames = bandnames).kFoldCV(10)
result

## Step 6: Model evaluation

### Compute average accuracy and summed confusion matrix for all folds

In [32]:
modeClassImage = prepareMetrics(classImage = result, nClasses = NCLASS, nFolds = FOLDS).confusionMatrix()
modeClassImage

The average accuracy is [1] across 10 folds


## Step 7: Visualise clasiified Image

In [44]:
legend_dict = {
    "Wattle": "00FF00",
    "Bracken": "008000",
    "Eucalyptus": "808080",
    "Forest": "800080",
    "Grassland": "FF00FF",
    "Mixed Woody Grassland": "00FFFF",
    "None Vegetated Features": "FFFF00",
    "Pines": "000080",
    "Recently Cleared Plantations/Young Pines": "800000"
}

palette = ['00FF00',# Wattle-Lime
          '008000', #Bracken-Green
          '808080', #Eucalyptus-Grey
          '800080', #Forest-Purple
          'FF00FF', #Grassland-Pink
          '00FFFF', #Mixed Woody Grassland-Aqua
          'FFFF00', #None Vegetated Features-Yellow
          '000080', #Pines-Navy
          '800000'] #(Recently Cleared Plantations/Young Pines)-Maroon (DarkBrown)

vis_params = {
    'min': 0,
    'max': 8,
    'palette': palette,
}

Map = geemap.Map()
Map.centerObject(study_area1, 8)
Map.addLayer(modeClassImage, vis_params, 'Classified Image')
Map.add_legend(legend_dict = legend_dict)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Step 8: Export classified image (optional)

In [ ]:
# export results
export_params3 = {
    'image': modeClassImage,
    'description': 'export_classImage',
    'assetId': 'users/geethensingh/classImage',
    'scale': 10,
    'region': study_area1
}

# Export the data to the Earth Engine asset
ee.batch.Export.image.toAsset(**export_params3).start()